# Bioimage Analysis Workflow 1 - Leaf Cellular Morphological Analysis (2)

## Import Data and necessary packages

In [ ]:
from skimage.io import imread

import matplotlib.pyplot as plt

import numpy as np

import scipy.ndimage as ndi

In [ ]:
# Define path
labels_filepath = r'example_data/cell_labels.tif'
img_filepath = r'example_data/Leaf_PM_Nuclei.tif'

# Read images
cell_labels = imread(labels_filepath)
img = imread(img_filepath)

# Split channel
img_ch0 = img[0,:,:]
img_ch1 = img[1,:,:]

# Define the x & y range for cropping
x_start = 500
x_end = 1500
y_start = 500
y_end = 1500

# Crop
img_ch0 = img_ch0[y_start:y_end,x_start:x_end]
img_ch1 = img_ch1[y_start:y_end,x_start:x_end]

# Display label on top of the image
plt.imshow(img_ch1, cmap = 'gray', interpolation= 'none')
plt.imshow(cell_labels, cmap = 'prism', interpolation= 'none', alpha = 0.3)

## Clean Edges

In [ ]:
# (i) Create an image border mask

# We need some way to check if a cell is at the border. For this, we generate a 'mask' of the image border,
# i.e. a Boolean array of the same size as the image where only the border pixels are set to `1` and all 
# others to `0`, like this:
#   1 1 1 1 1
#   1 0 0 0 1
#   1 0 0 0 1
#   1 0 0 0 1
#   1 1 1 1 1
# There are multiple ways of generating this mask, for example by erosion or by array indexing.
# It is up to you to find a way to do it. (Hint: one of the the easiest ways to do this is via scipy.ndimage.binary_dilation.
# check the parameter "border_value")

border_mask = np.zeros(cell_labels.shape, dtype=bool)
border_mask = ndi.binary_dilation(border_mask, border_value=1)

In [ ]:
# (ii) 'Delete' the cells at the border

# 1) Find the cell ROIs that are crossing the border of the image

# Find the border ROI IDs, by first multiply the border_mask by the segmentation mask
border_mask_rois = border_mask * cell_labels

# Then get an array of ROI IDs by finding the unique elements in the array
border_roi_ids = np.unique(border_mask_rois)
border_roi_ids

In [ ]:
# 2) 'Delete' ROIs by their IDs

# Create a copy of the segmentation with np.copy()
clean_cell_labels = np.copy(cell_labels)

# Iterate over ROI IDs on the border and set the those ROIs to background (0)
for roi_id in border_roi_ids:
    
    # Create a mask that contains only the 'current' ROI of the iteration
    roi_mask = cell_labels == roi_id
    
    # Set the position of that roi_mask to background (zero) in the clean_seg
    clean_cell_labels[roi_mask] = 0

In [ ]:
plt.imshow(clean_cell_labels, cmap = 'gray')

## Extracting Quantitative Measures

In [ ]:
# Use the function 'regionprops_table' from the skimage.measure module
from skimage.measure import regionprops_table

# Obtain measurement and save in a parameter
props = regionprops_table(clean_cell_labels, img_ch0, properties=[
                                                                'label',
                                                                'area', 
                                                              'intensity_mean',
                                                              'eccentricity',
                                                              'feret_diameter_max',
                                                              'perimeter',
                                                              'solidity',
                                                             ])

In [ ]:
# Convert props to a pandas dataframe
import pandas as pd

props_df = pd.DataFrame(props)

props_df.head()

In [ ]:
# Save data as csv
props_df.to_csv("leaf_measurement.csv")

## Generate plots about the data

In [ ]:
# Create a histogram of the cell area
plt.figure(figsize=(4,3))
plt.hist(props['area'], bins = 50)
plt.xlabel('Cell Area [pixel]')
plt.ylabel('Count')

In [ ]:
# Create a scatter plot of cell solidity over perimeter
plt.figure(figsize=(5,5))
plt.scatter(props['solidity'],props['perimeter'], edgecolor='k', s=30, alpha=0.5)
plt.xlabel('solidity')
plt.ylabel('Perimeter')

In [ ]:
# Get number of cells
num_cells = props['label'].shape[0]

# Create a heat map of cell solidity
heat_map = np.zeros_like(clean_cell_labels, dtype = np.uint8)

# Get minimum value
min_measure = min(props['solidity'])

# Get maximum value
max_measure = max(props['solidity'])

# Iterate over ROI IDs
for cell_id in range(num_cells):

    # Iterate over ROI IDs on the border and set the those ROIs to background (0)
    cell_mask = clean_cell_labels == props['label'][cell_id]

    # Convert the cell ID to 8 bit
    measure_8bit = (props['solidity'][cell_id]-min_measure)*255/(max_measure - min_measure)

    # Assign value to heat map
    heat_map[cell_mask] = measure_8bit
    

In [ ]:
plt.figure(figsize = (5,5))

# Show the heat map. Use a suitable colormap
plt.imshow(heat_map, cmap = 'PRGn')

# Save image as png
plt.savefig('Heatmap.png')